In [2]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [ ]:
def set_seed(seed=0):
    torch.manual_seed(seed)
    np.random_seed(seed)
    torch.cuda.manual_seed_all(seed)